In [1]:
!git clone https://github.com/herm23/Project-NNDL.git

Cloning into 'Project-NNDL'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 67 (delta 14), reused 62 (delta 12), pack-reused 0 (from 0)
Receiving objects: 100% (67/67), 3.25 MiB | 31.07 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [2]:
%cd Project-NNDL/

/content/Project-NNDL


Link per il datset di CompCar (trovato su sito ufficiale): https://drive.google.com/drive/folders/18EunmjOJsbE5Lh9zA0cZ4wKV6Um46dkg


In [3]:
!pip install -U -q PyDrive

Import necessari

In [4]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

Scarico i dati direttamente dal drive pubblico

In [5]:
#Autenticazione
auth.authenticate_user()

gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [6]:
# Estrazione dei dati dalla cartella
# ID della cartella pubblica
folder_id = '18EunmjOJsbE5Lh9zA0cZ4wKV6Um46dkg'

# Cartella locale per salvare i dati
save_path = '/content/Project-NNDL/compcar_dataset'
os.makedirs(save_path, exist_ok=True)

# Lista file nella cartella Drive
file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()

# Scarica tutti i file
for file in file_list:
    print(f"Scaricando {file['title']}")
    file.GetContentFile(os.path.join(save_path, file['title']))



Scaricando sv_data.zip
Scaricando sv_data.z03
Scaricando sv_data.z02
Scaricando sv_data.z01
Scaricando data.zip
Scaricando data.z22
Scaricando data.z21
Scaricando data.z20
Scaricando data.z19
Scaricando data.z18
Scaricando data.z17
Scaricando data.z16
Scaricando data.z15
Scaricando data.z14
Scaricando data.z13
Scaricando data.z12
Scaricando data.z11
Scaricando data.z10
Scaricando data.z09
Scaricando data.z08
Scaricando data.z07
Scaricando data.z06
Scaricando data.z05
Scaricando data.z04
Scaricando data.z03
Scaricando data.z02
Scaricando data.z01
Scaricando README.txt


In [7]:
!apt-get install -y p7zip-full


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [8]:
%cd /content/Project-NNDL/compcar_dataset

/content/Project-NNDL/compcar_dataset


In [9]:
!7z x data.zip -o./compcar_dataset/unzipped


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,12 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (50657),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 391417395 bytes (374 MiB)

Extracting archive: data.zip
  1% 4096 Open               89% 294912 Open                --
Path = data.zip
Type = zip
Physical Size = 391417395
Embedded Stub Size = 4
64-bit = +
Total Physical Size = 16539487795
Multivolume = +
Volume Index = 22
Volumes = 23

  0%    
Enter password (will not be echoed):
  0% 5 - data/image/1/1101/2011/07b90decb92ba6.jpg                                                    0% 144 - data/image/1/1112/2013/16179e5d599552.jpg                  

Funzioni che uso lungo il progetto

In [57]:
from sklearn.model_selection import train_test_split
from collections import defaultdict
import statistics

def readFile(path):
  with open(path, 'r') as f:
    items = [line.strip() for line in f.readlines()]

  return items

#mescolamento casuale
def shuffleElems(path):
  items = readFile(path)
  train_data, val_data = train_test_split(items, test_size=0.2, random_state=42)

  return train_data, val_data

#mescolamento non causale ma mettendo nel set di validation tutte le classi che ci sono in training
def compcar_split_test_valid(path, valid_perc):
  items = readFile(path)
  groups = defaultdict(list)
  for item in items:
      key = item.split('/', 1)[0]
      groups[key].append(item)

  # 3. Statistiche di base
  # num_groups = len(groups)
  # avg_length = statistics.mean(len(g) for g in groups.values()) if groups else 0
  # print(f"Numero di gruppi: {num_groups}")
  # print(f"Lunghezza media dei gruppi: {avg_length:.2f}")

  # 4. Separa 20% e 80% per gruppo
  valid_list = []
  train_list = []
  # print("Num gruppi:", len(groups))
  for g in groups.values():
      n = len(g)
      cutoff = int(n * valid_perc)    # primi floor(20%) elementi
      valid_list.extend(g[:cutoff])
      train_list.extend(g[cutoff:])

  # 5. Stampa i primi 100 elementi di ciascuna lista
  # print("\nPrimi 100 elementi della lista 20%:")
  # for elem in valid_list[:100]:
  #     print(elem)
  # print(f"(... su {len(valid_list)} totali)")

  # print("\nPrimi 100 elementi della lista 80%:")
  # for elem in train_list[:100]:
  #     print(elem)
  # print(f"(... su {len(train_list)} totali)")

  # 6. Ritorna le due liste
  return train_list, valid_list

Test delle funzioni

In [50]:
# compcar_split_test_valid("/content/Project-NNDL/compcar_dataset/compcar_dataset/unzipped/data/train_test_split/classification/train.txt", 0.2)
# compcar_split_test_valid("/content/Project-NNDL/compcar_dataset/compcar_dataset/unzipped/data/train_test_split/classification/test.txt", 0.2)

**Creazione del dataset (seguendo modello prof lavoratorio 4)**

In [58]:
import os
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image

class CompCarsDataset(Dataset):
    def __init__(self, root_dir, txt_path=None, inputs_list=None, transform=None):
        self.transform = transform
        self.root_dir = root_dir
        #Lettura immagini
        if txt_path is not None:
          self.images = readFile(txt_path)
        else:
          self.images = inputs_list

        #Corrispondendi marche
        self.makes_list = [p.split('/')[0] for p in self.images]

        #dizionario usato per associare ad ogni marca un indice
        #l'indice è necessario per la loss di Cross Entropy
        self.make_dict = {make: idx for idx, make in enumerate(sorted(set(self.makes_list)))}

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        # print("Hai richiamato get item")
        full_path = os.path.join(self.root_dir, self.images[idx])
        image = Image.open(full_path).convert('RGB')
        make = self.make_dict[self.makes_list[idx]] #estraggo prima il nome del modello e poi ritorno il suo indice corrispondente

        if self.transform:
            image = self.transform(image)

        return image,make

**Divisione di train e test basandosi sul file presente nella cartella train_test_split
**

In [64]:
from torch.utils.data import DataLoader, random_split
from torch.utils.data import Subset
from torchvision.transforms import Compose, Resize, RandomAffine, RandomHorizontalFlip, ToTensor
import torch
import os
import pandas as pd
import h5py
import random
import numpy as np
import matplotlib.pyplot as plt


# Percorsi
root_dir = "/content/Project-NNDL/compcar_dataset/compcar_dataset/unzipped/data/image"
train_txt = "/content/Project-NNDL/compcar_dataset/compcar_dataset/unzipped/data/train_test_split/classification/train.txt"
test_txt = "/content/Project-NNDL/compcar_dataset/compcar_dataset/unzipped/data/train_test_split/classification/test.txt"

#params
valid_perc = 0.2
batch_size = 32
epochs=30
#usati nell'optimizer
learning_rate = 1e-2
w_decay = 0


# Rendo invariandt a modifiche
# transform = Compose([
#     Resize((224, 224)),
#     RandomAffine(degrees=5),
#     RandomHorizontalFlip(),
#     ToTensor()
# ])
transform = Compose([
    Resize((224,224)),
    ToTensor()
])

# Dataset (effettuo questa operazione perche mi serve avere il validation e training separati)
# Divisione dei 2 set scartata perchè
# train_list, valid_list = shuffleElems(train_txt)
train_list, valid_list = compcar_split_test_valid(train_txt, valid_perc)

# print("Primi 100 elementi di train: ", train_list[200:301])
# print("Primi 100 elementi di valid: ", valid_list[200:301])

train_dataset = CompCarsDataset(root_dir, None, train_list, transform=transform)
valid_dataset = CompCarsDataset(root_dir, None, valid_list, transform=transform)
test_dataset = CompCarsDataset(root_dir, test_txt, None, transform=transform)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# print di controllo
# print(train_loader[0])

# Rete Resnet 50

Definizione dei blocchi

In [65]:
from torch.nn import Module, Sequential, Conv2d, BatchNorm2d, ReLU

class MainPath(Module):

    def __init__(self, in_channels, filters, kernel_size, stride=1):
        super().__init__()
        F1, F2, F3 = filters
        self.main_path = Sequential(
            Conv2d(in_channels, F1, kernel_size=1, stride=stride),
            BatchNorm2d(F1),
            ReLU(),
            Conv2d(F1, F2, kernel_size=kernel_size, padding=kernel_size//2),
            BatchNorm2d(F2),
            ReLU(),
            Conv2d(F2, F3, kernel_size=1),
            BatchNorm2d(F3),
        )
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()
        if isinstance(module, torch.nn.Conv2d):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()

    def forward(self, x):
        y = self.main_path(x)
        return y

class IdentityBlock(MainPath):

    def __init__(self, in_channels, filters, kernel_size):
        super().__init__(in_channels, filters, kernel_size)
        self.relu = ReLU()

    def forward(self, x):
        y = self.relu(self.main_path(x) + x)
        return y

In [66]:
class ConvolutionalBlock(MainPath):

    def __init__(self, in_channels, filters, kernel_size):
        super().__init__(in_channels, filters, kernel_size, stride=2)
        self.relu = ReLU()
        self.shortcut_path = Sequential(
            Conv2d(in_channels, filters[2], kernel_size=1, stride=2),
            BatchNorm2d(filters[2])
        )
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()
        if isinstance(module, torch.nn.Conv2d):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()

    def forward(self, x):
        y = self.relu(self.main_path(x) + self.shortcut_path(x))
        return y

Definizione rete

In [67]:
from torch.nn import MaxPool2d, AvgPool2d, Linear, Dropout
class ResNet50(Module):

    def __init__(self):
        super().__init__()
        self.network = Sequential(
            Conv2d(3, 64, kernel_size=7, stride=2),
            BatchNorm2d(64),
            MaxPool2d(kernel_size=3, stride=2),
            ConvolutionalBlock(64, [64, 64, 256], kernel_size=3),
            Dropout(0.2),
            IdentityBlock(256, [64, 64, 256], kernel_size=3),
            IdentityBlock(256, [64, 64, 256], kernel_size=3),
            ConvolutionalBlock(256, [128, 128, 512], kernel_size=3),
            Dropout(0.2),
            IdentityBlock(512, [128, 128, 512], kernel_size=3),
            IdentityBlock(512, [128, 128, 512], kernel_size=3),
            IdentityBlock(512, [128, 128, 512], kernel_size=3),
            ConvolutionalBlock(512, [256, 256, 1024], kernel_size=3),
            Dropout(0.2),
            IdentityBlock(1024, [256, 256, 1024], kernel_size=3),
            IdentityBlock(1024, [256, 256, 1024], kernel_size=3),
            IdentityBlock(1024, [256, 256, 1024], kernel_size=3),
            IdentityBlock(1024, [256, 256, 1024], kernel_size=3),
            IdentityBlock(1024, [256, 256, 1024], kernel_size=3),
            ConvolutionalBlock(1024, [512, 512, 2048], kernel_size=3),
            Dropout(0.2),
            IdentityBlock(2048, [512, 512, 2048], kernel_size=3),
            IdentityBlock(2048, [512, 512, 2048], kernel_size=3),
            AvgPool2d(kernel_size=2, stride=2)
        )
        #75 è il numero totale di classi presenti nello split train e test che è stato fornito (make)
        # self.classification_layer = Linear(2048, 75)
        self.classification_layer = Linear(8192, 75)
        # self.classification_layer = Linear(self._get_conv_output_dim(), 75)
        self.apply(self._init_weights)

    def forward(self, x):
        y = self.network(x).reshape((x.shape[0], -1))
        # print("shape dopo la resnet 50:", y.shape) # Add this line to print the shape
        y = self.classification_layer(y)
        return y

    def _init_weights(self, module):
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()
        if isinstance(module, torch.nn.Conv2d):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()

Train + valid

In [68]:
from torch.optim import SGD, Adam
from torch.nn import CrossEntropyLoss
from tqdm import tqdm
model = ResNet50()
# opt = SGD(model.parameters(), lr=1e-2, weight_decay = 0)
opt = Adam(model.parameters(), lr=learning_rate, weight_decay=w_decay)
loss_fn = CrossEntropyLoss()
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
best_val = np.inf
for epoch in range(epochs):
    model.train()
    print(f"Epoch: {epoch+1}")
    iterator = tqdm(train_loader)
    for batch_x, batch_y in iterator:
        batch_x = batch_x.to(device)
        # print("stampa batch_y: ", batch_y)
        batch_y = batch_y.to(device)

        y_pred = model(batch_x)

        loss = loss_fn(y_pred, batch_y)

        opt.zero_grad()
        loss.backward()
        opt.step()
        iterator.set_description(f"Train loss: {loss.detach().cpu().numpy()}")

    model.eval()
    with torch.no_grad():
        predictions = []
        true = []
        for batch_x, batch_y in tqdm(val_loader):
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)

            y_pred = model(batch_x)

            predictions.append(y_pred)
            true.append(batch_y)
        predictions = torch.cat(predictions, axis=0)
        true = torch.cat(true, axis=0)
        val_loss = loss_fn(predictions, true)
        # val_acc = (torch.sigmoid(predictions).round() == true).float().mean()
        # print(f"loss: {val_loss}, accuracy: {val_acc}")
        print(f"loss: {val_loss}")

    if val_loss < best_val:
        print("Saved Model")
        torch.save(model.state_dict(), "model.pt")
        best_val = val_loss



Epoch: 1


100%|██████████| 100/100 [00:26<00:00,  3.77it/s]


loss: 3.847548723220825
Saved Model
Epoch: 2


100%|██████████| 100/100 [00:26<00:00,  3.75it/s]


loss: 3.8423497676849365
Saved Model
Epoch: 3


...

loss: 3.8121745586395264
Saved Model
Epoch: 9


100%|██████████| 100/100 [00:25<00:00,  3.92it/s]


loss: 3.8553993701934814
Epoch: 10


Train loss: 3.792515516281128:   6%|▌         | 24/402 [00:07<01:53,  3.34it/s]


KeyboardInterrupt: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Valutazione rete

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_curve, roc_auc_score

def evaluate_network(dataloader, model, data_split):
    model.eval()
    with torch.no_grad():
        predictions = []
        true = []
        for batch_x, batch_y in tqdm(dataloader):
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)

            y_pred = model(batch_x)

            predictions.append(y_pred)
            true.append(batch_y)
        predictions = torch.cat(predictions, axis=0)
        true = torch.cat(true, axis=0)
        loss = loss_fn(predictions, true).detach().cpu().numpy()
        predictions = torch.sigmoid(predictions).detach().cpu().numpy()
        true = true.detach().cpu().numpy()

        fpr, tpr, thresholds = roc_curve(true, predictions)
        auc = roc_auc_score(true, predictions)
        predictions = predictions.round()
        precision, recall, fscore, _= precision_recall_fscore_support(true, predictions, average='binary')
        accuracy = accuracy_score(true, predictions)

        print(f"{data_split} loss: {loss}, accuracy: {accuracy}, precision: {precision}, recall: {recall}, f1: {fscore}, roc_auc: {auc}")

        plt.figure()
        plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % auc)
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f'{data_split} receiver operating characteristic (ROC)')
        plt.legend(loc="lower right")

In [ ]:
model.load_state_dict(torch.load("model.pt"))
evaluate_network(train_loader, model, "Training Dataset")
evaluate_network(val_loader, model, "Validation Dataset")
evaluate_network(test_loader, model, "Test Dataset")